In [2]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

In [3]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 1


In [4]:
df = pd.read_csv("all_data_clean.csv",header=None,
                 names=['timestamp','accel_x','accel_y','accel_z','gyro_x','gyro_y','gyro_z','label'])
df = df.dropna()
df = df.reset_index(drop=True)
df = df[1:]
df.shape

C:\Users\siyua\AppData\Local\Temp\ipykernel_7580\3893055260.py:1: DtypeWarning: Columns (1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("all_data_clean.csv",header=None,


(125699, 8)

In [16]:
features = ['accel_x','accel_y','accel_z','gyro_x','gyro_y','gyro_z']
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])
window_size = 50
step_size = 2

X = []
y = []

for start in range(0,len(df) - window_size, step_size):
    end = start + window_size
    window = df.iloc[start:end]
    label = window['label'].mode()[0]
    
    X.append(window[features].values)
    y.append(label)

X = np.array(X)
y = np.array(y)
print("window data shape",X.shape,y.shape)

window data shape (62825, 50, 6) (62825,)


In [17]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_cat = to_categorical(y_encoded)

print("Label classes:", encoder.classes_)

Label classes: ['falling' 'sitting' 'standing' 'walking']


In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(
    X, y_cat, test_size = 0.2, random_state=42, stratify=y_cat
)

In [19]:
#define 1D CNN Model 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,MaxPooling1D,LSTM,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv1D(16, 3, activation='relu', input_shape=(window_size, 6)),
    MaxPooling1D(2),
    Conv1D(32, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(y_cat.shape[1], activation='softmax')
])
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer= optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 48, 16)            304       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 24, 16)           0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 22, 32)            1568      
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 11, 32)           0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 352)               0         
                                                                 
 dense_4 (Dense)             (None, 32)               

In [20]:
#Train the model
with tf.device('/GPU:0'):
    history = model.fit(X_train,y_train,epochs=100,batch_size=64,
                    validation_data=(X_test,y_test))

Epoch 1/100
786/786 [==============================] - 3s 3ms/step - loss: 1.2357 - accuracy: 0.4019 - val_loss: 1.1285 - val_accuracy: 0.4884
Epoch 2/100
786/786 [==============================] - 2s 2ms/step - loss: 1.0516 - accuracy: 0.5342 - val_loss: 1.0387 - val_accuracy: 0.5404
Epoch 3/100
786/786 [==============================] - 2s 2ms/step - loss: 0.9866 - accuracy: 0.5718 - val_loss: 0.9779 - val_accuracy: 0.5775
Epoch 4/100
786/786 [==============================] - 2s 2ms/step - loss: 0.9457 - accuracy: 0.5934 - val_loss: 0.9453 - val_accuracy: 0.5954
Epoch 5/100
786/786 [==============================] - 2s 2ms/step - loss: 0.9173 - accuracy: 0.6066 - val_loss: 0.9196 - val_accuracy: 0.6078
Epoch 6/100
786/786 [==============================] - 2s 2ms/step - loss: 0.8919 - accuracy: 0.6203 - val_loss: 0.9043 - val_accuracy: 0.6139
Epoch 7/100
786/786 [==============================] - 2s 2ms/step - loss: 0.8737 - accuracy: 0.6286 - val_loss: 0.8907 - val_accuracy: 0.6208

In [21]:
#Evaluate the model
loss, acc = model.evaluate(X_test,y_test)
print("Test accuracy",acc)

393/393 [==============================] - 1s 2ms/step - loss: 0.9453 - accuracy: 0.6528
Test accuracy 0.6527656316757202


In [11]:
model2 = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(window_size, 6)),
    MaxPooling1D(2),
    Dropout(0.3),
    LSTM(64 ,return_sequences=False),
    Dropout(0.3),
    Dense(64,activation='relu'),
    Dense(y_cat.shape[1], activation='softmax')
])

optimizer = Adam(learning_rate=0.0005)
model2.compile(optimizer= optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 198, 64)           1216      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 dropout (Dropout)           (None, 99, 64)            0         
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                      

In [12]:
#model2 trianing 
# with tf.device('/GPU:0'):
#     history = model2.fit(X_train,y_train,epochs=100,batch_size=64,
#                     validation_data=(X_test,y_test))

In [13]:
# loss, acc = model2.evaluate(X_test,y_test)
# print("Test accuracy",acc)

In [14]:
def representative_dataset():
    for _ in range(100):
        data = np.random.rand(62750, 200, 6).astype(np.float32)  # shape must match your model input
        yield [data]

In [22]:
#convert to TFLITE


converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.target_spec.supported_ops = [tf.float32]



tflite_model = converter.convert()


with open("fall_detector_cnn_Fl32.tflite","wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\siyua\AppData\Local\Temp\tmpre9bjdvy\assets


INFO:tensorflow:Assets written to: C:\Users\siyua\AppData\Local\Temp\tmpre9bjdvy\assets
